The New York Neighborhood Recommender

I. Purpose

Just imagine finding a well payed job in New York. Finally having this grand and diverse city as yo
ur new home. As many people found out, it can also be tricky to go there. What if you feel strange? Or you find out you don't like your Neighborhood? This App will help you getting along by finding the right Neighborhood to move to based on where you live now. So you won't feel alone and strange. For anyone renting apartments it can be profitable as well. People find the right place to call home faster and are more happy, thus they will stay longer. Plus they will recommend you.

II. Data Acquisition

We will use data based on the Foursquare API. With it we can gather information on all revenues in a given Neighborhood of New Yor. We will also gather the same information for the place you are currently living in.

Lets start! First with the New York Data. We downloaded a dataset with all Neighborhoods and geolocations.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [17]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()

(306, 3)

Now get the hottest venues.

In [3]:
CLIENT_ID = '3A1JPR4E42ITJEPNWTDJ0AYIDDLM4ULERLLRFAUKHXDQIM3L' # your Foursquare ID
CLIENT_SECRET = 'V0ZROHHBYCLH4A2ORB4Y1EIIQNNK3OFHBV4L0ZY1WXTHMGSZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 5 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
        
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        #print(requests.get(url).json())
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',                   
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [5]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [18]:
newyork_venues.shape

(1474, 7)

In [46]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()

print(newyork_grouped.shape)


(301, 248)


In [47]:
#print(neighborhoods.sort_values(['Neighborhood']))
print(newyork_grouped.set_index('Neighborhood').join(neighborhoods.set_index('Neighborhood')).shape)




(305, 249)


In [7]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [8]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Donut Shop,Pizza Place,Fast Food Restaurant,Dessert Shop,Martial Arts Dojo,Eastern European Restaurant,Filipino Restaurant,Field,Farmers Market,Farm
1,Annadale,Sports Bar,Train Station,Pet Store,Diner,Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space
2,Arden Heights,Pizza Place,Pharmacy,Coffee Shop,Playground,Cycle Studio,Cupcake Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
3,Arlington,Scenic Lookout,Grocery Store,Boat or Ferry,American Restaurant,Bus Stop,Falafel Restaurant,Electronics Store,Empanada Restaurant,Event Space,Factory
4,Arrochar,Italian Restaurant,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant,Eastern European Restaurant,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm


Now the data for your location. I chose my home as an example. Later in the analysis we can see which New York Neighborhoods are similar bases on the data collected.

In [22]:
home_name = ['Eimsbüttel']
home_lat = ['53.555464']
home_lon = ['9.811365']

home_venues = getNearbyVenues(names=home_name,
                                   latitudes=home_lat,
                                   longitudes=home_lon
                                  )
# one hot encoding with the same columns
home_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
home_onehot['Neighborhood'] = home_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [home_onehot.columns[-1]] + list(home_onehot.columns[:-1])
home_onehot = home_onehot[fixed_columns]

home_grouped = home_onehot.groupby('Neighborhood').mean().reset_index()

#print(home_grouped.head())

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
home_venues_sorted = pd.DataFrame(columns=columns)
home_venues_sorted['Neighborhood'] = home_grouped['Neighborhood']

for ind in np.arange(home_grouped.shape[0]):
    home_venues_sorted.iloc[ind, 1:] = return_most_common_venues(home_grouped.iloc[ind, :], num_top_venues)

home_grouped_clustering = home_grouped.drop('Neighborhood', 1)

home_venues_sorted.shape

(1, 11)

In [45]:
# set number of clusters
kclusters = 5

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

print(newyork_grouped.shape)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

print(kmeans.cluster_centers_.size)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10]) 

newyork_merged = neighborhoods

print("newyork_grouped_clustering.size")
print(newyork_grouped.head())
print("newyork_merged.size")
print(newyork_merged.shape)
print("kmeans.labels_.siz")
print(kmeans.labels_.head())

print(newyork_merged.head()) # check the last columns!


# add clustering labels
newyork_merged['Cluster Labels'] = kmeans.labels_


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


print(kmeans.predict(home_grouped))

(301, 248)
1235
[4 0 4 0 1 3 0 1 1 2]
newyork_grouped_clustering.size
    Neighborhood  Yoga Studio  Accessories Store  Adult Boutique  \
0       Allerton          0.0                0.0             0.0   
1       Annadale          0.0                0.0             0.0   
2  Arden Heights          0.0                0.0             0.0   
3      Arlington          0.0                0.0             0.0   
4       Arrochar          0.0                0.0             0.0   

   Afghan Restaurant  African Restaurant  American Restaurant  Antique Shop  \
0                0.0                 0.0                  0.0           0.0   
1                0.0                 0.0                  0.0           0.0   
2                0.0                 0.0                  0.0           0.0   
3                0.0                 0.0                  0.2           0.0   
4                0.0                 0.0                  0.0           0.0   

   Arcade  Arepa Restaurant  Argentinian Resta

AttributeError: 'numpy.ndarray' object has no attribute 'head'